In [1]:
import numpy as np
from astropy.coordinates import SkyCoord, HeliocentricTrueEcliptic, ITRS, EarthLocation
import torch as tn
from time import sleep
from datetime import datetime, timezone

In [2]:
CL = tn.tensor([
    [0, 0, 0, 0],
    [0, 1e-5, 0, 0],
    [0, 0, 1e-5, 0],
    [0, 0, 0, 2e-5]
], dtype=tn.float32)

CR = tn.tensor([
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
], dtype=tn.float32)

p1 = 1 / np.sqrt(2) * tn.tensor([1,0, 1,1], dtype=tn.float32)

p2 = 1 / np.sqrt(2) * tn.tensor([1,1, 0,-1], dtype=tn.float32)

In [3]:

tn.set_printoptions(precision=10, sci_mode=False)


# Define the location of CMS in terms of longitude, latitude and azimuth

azimuth = 1.7677    
latitude = 0.8082  
longitude = 0.1061   

# Define the Earth's angular velocity (rad/s)
omega_utc = 7.27221e-5     # Earth's angular velocity in rad/s at UTC.


# omega_t = omega_utc*time + 3.2830    #omeag*t = omega_UTC * t_CMS +phi_UNIX +phi_longit

# Rotation matrices to go from ITRS to CMS frame

# rotation around the z-axis which makes the x-axis normal to the plane of the LHC.

R_z = tn.tensor([
        [1,0,0,0],
        [0,0,-1,0],
        [0,1,0,0],
        [0,0,0,1]
    ], dtype=tn.float32)


# To orient the z-axis towards the North. We rotate counterclockwise around the x′ axis with an angle π −θ (co-azimuth).
def R_x(angle):
    return tn.tensor([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, - np.cos(angle), np.sin(angle)],
        [0, 0, - np.sin(angle), - np.cos(angle)]
    ], dtype=tn.float32)


# Rotation around the y axis to align the z-axis with the Z-axis of the SCF.
def R_y(angle):
    return tn.tensor([
        [1, 0, 0, 0],
        [0, np.cos(angle), 0, np.sin(angle)],
        [0, 0, 1, 0],
        [0, -np.sin(angle), 0, np.cos(angle)]
    ], dtype=tn.float32)


# A final rotation around the Z-axis has two purposes: to follow the rotation of the Earth over time and to synchronize with the SCF:
def R_Z(angle):
    return tn.tensor([
        [1, 0 , 0, 0],
        [0, np.cos(angle), -np.sin(angle), 0],
        [0, np.sin(angle), np.cos(angle), 0],
        [0, 0, 0, 1]
    ], dtype=tn.float32)


## test

In [5]:
# Define the time duration (60 seconds)
duration = 60
end_time = datetime.now(timezone.utc).timestamp() + duration

while datetime.now(timezone.utc).timestamp() < end_time:
    # current time in UTC (CMS)
    current_time = datetime.now(timezone.utc)
    time_utc = current_time.timestamp()
    
    # Calculate omega_t
    omega_t = omega_utc * time_utc + 3.2830  # omega*t = omega_UTC * t_CMS + phi_UNIX + phi_longit

    # Construct the complete rotation matrix from SCF to CMS
    R_matrix = R_y(latitude) @ R_x(azimuth) @ R_z @ R_Z(omega_t)
    contr = tn.matmul(CL, R_matrix)
    result = tn.dot(p1, tn.mv(contr, p2))

    # Print the rotation matrix and flush the output
    print(R_matrix)
    print(result)
    print(flush=True)
    
    # Wait for 4 seconds
    sleep(4)

tensor([[ 1.0000000000,  0.0000000000,  0.0000000000,  0.0000000000],
        [ 0.0000000000, -0.9863933921,  0.0837820172,  0.1414520144],
        [ 0.0000000000,  0.1280711442, -0.1478863060,  0.9806770086],
        [ 0.0000000000,  0.1030819416,  0.9854492545,  0.1351440251]])
tensor(    -0.0000045837)

tensor([[ 1.0000000000,  0.0000000000,  0.0000000000,  0.0000000000],
        [ 0.0000000000, -0.9863690138,  0.0840691626,  0.1414520144],
        [ 0.0000000000,  0.1280280799, -0.1479235888,  0.9806770086],
        [ 0.0000000000,  0.1033688188,  0.9854192734,  0.1351440251]])
tensor(    -0.0000045810)

tensor([[ 1.0000000000,  0.0000000000,  0.0000000000,  0.0000000000],
        [ 0.0000000000, -0.9863444567,  0.0843564570,  0.1414520144],
        [ 0.0000000000,  0.1279849857, -0.1479608715,  0.9806770086],
        [ 0.0000000000,  0.1036558151,  0.9853889942,  0.1351440251]])
tensor(    -0.0000045783)

tensor([[ 1.0000000000,  0.0000000000,  0.0000000000,  0.0000000000],
      